In [1]:
import numpy as np
import tensorflow as tf
import copy
import time

from joblib import Parallel, delayed
import multiprocessing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.preprocessing import StandardScaler

from evolution import model

In [2]:
featuremat = np.load('data/training_data/featurematrix.npy')
time_to_failure_vec = np.load('data/training_data/time_to_failure.npy')
testmat = np.load('data/test_processed/featurematrix_test.npy')

In [3]:
trainset = np.random.choice(np.arange(len(time_to_failure_vec)),size=int(len(time_to_failure_vec)*.9),replace=False)
validationset = list(set(np.arange(len(time_to_failure_vec)))-set(trainset))

In [4]:
trainmat = featuremat[trainset,:]
validationmat = featuremat[validationset,:]

time_to_failure_train = time_to_failure_vec[trainset]
time_to_failure_validation = time_to_failure_vec[validationset]

## Normalise data

In [5]:
scaler = StandardScaler()
scaler.fit(trainmat)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [6]:
trainmat_scale = scaler.transform(trainmat)
validationmat_scale = scaler.transform(validationmat)

# Evolution

In [7]:
generations = 100
pop_size = 100
epochs = 100
batch_size = 64
num_all_features = 33

In [8]:
model_list = []
for i in range(pop_size):
    model_list.append(model({np.random.randint(num_all_features)},[np.random.randint(1,17)],[0],num_all_features))

In [9]:
def train_model(model_list,trainmat,validationmat,model_ind):
    
    trainmat_use = trainmat[:,list(model_list[model_ind].featureset)]
    validationmat_use = validationmat[:,list(model_list[model_ind].featureset)]

    m = model_list[model_ind].build()
    
    opt = tf.keras.optimizers.Adam(lr=0.1,decay=.1)
    m.compile(optimizer=opt,
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])
    hist = m.fit(trainmat_use, 
          time_to_failure_train, 
          validation_data=(validationmat_use,time_to_failure_validation), 
          epochs=epochs, 
          batch_size=batch_size,
          verbose=0
         )
        
    loss = hist.history['val_mean_absolute_error'][-1]
    return loss

In [ ]:
num_cores = multiprocessing.cpu_count()

In [ ]:
loss = np.zeros([generations,pop_size])
models_employed = []
for gen in range(generations):
    t = time.time()
    
    models_employed.append(model_list) 
    
    loss_generation = Parallel(n_jobs=num_cores)(
            delayed(train_model)(model_list,trainmat,validationmat,model_ind) for model_ind in range(pop_size)
    )
    loss[gen,:] = loss_generation
    
    tf.keras.backend.clear_session()
        
    live = np.argsort(loss[gen,:])[0:int(pop_size/2)]
    
    model_list2 = []
    for index in live:
        model_list2.append(model_list[index])
        m = copy.deepcopy(model_list[index])
        m.mutate()
        model_list2.append(m)

    model_list = model_list2

    print('Generation %d finished, %.3f sec, average loss %.3f, average loss of survivors %.3f' 
          %(gen, time.time()-t, np.mean(loss[gen,:]), np.mean(loss[gen,live])))

Generation 0 finished, 167.755 sec, average loss nan, average loss of survivors 5.148


/Users/djustus/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Generation 1 finished, 165.772 sec, average loss nan, average loss of survivors 4.817
Generation 2 finished, 167.404 sec, average loss 185.046, average loss of survivors 4.570
Generation 3 finished, 173.420 sec, average loss 5095626.634, average loss of survivors 4.620
Generation 4 finished, 179.478 sec, average loss 9829.025, average loss of survivors 4.137
Generation 5 finished, 188.623 sec, average loss nan, average loss of survivors 3.588
Generation 6 finished, 168.147 sec, average loss 875570.951, average loss of survivors 3.119
Generation 7 finished, 172.553 sec, average loss 4.078, average loss of survivors 2.378
Generation 8 finished, 177.523 sec, average loss 3.228, average loss of survivors 2.256
Generation 9 finished, 174.496 sec, average loss 2.996, average loss of survivors 2.255
Generation 10 finished, 169.309 sec, average loss 3.348, average loss of survivors 2.255
Generation 11 finished, 177.362 sec, average loss 3.243, average loss of survivors 2.255
Generation 12 fini

In [ ]:
for i in range(20):
    print(models_employed[9][i].num_layers)